In [1]:
import random
from firebase_admin import initialize_app, credentials, db

In [2]:
cred = credentials.Certificate('creds.json')

In [3]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

In [17]:
h = "Holz"
l = "Lehm"
g = "Getreide"
w = "Wolle"
e = "Erz"

In [9]:
seed = random.randrange(2 ** 32)
seed

1859272767

In [15]:
players = ['Simon', 'Dave', 'Heiko', 'Niko']
random.shuffle(players)

In [18]:
#ports = ['*', w, '*', e, '*', g, l, '*', h]
db.reference("test/initial").set({
    'seed': seed,
    'players': players,
    'tiles':  [[h, w, w], [g, e, g, h], [h, l, None, e, g], [g, e, h, w], [l, w, l]],
    'rolls': [[6, 3, 8], [2, 4, 5, 10], [5, 9, None, 6, 9], [10, 11, 3, 12], [8, 4, 11]],
})

In [ ]:
# shortcuts

def info():
    
    infos = players_ref.get()
    for info in infos.values():
        name = player_names[info['index']]
        resources = ' '.join(info.get('resources', []))
        cards = ' '.join(info.get('cards', []))
        print(f"{name}: {resources} | {cards}")
        
              
def give(index, gains):
    
    resources = res_refs[index].get() or []
    resources += gains
    resources = sorted(resources, key=lambda r: resource_order[r])
    res_refs[index].set(resources)
    

def take(index, costs):
    
    resources = res_refs[index].get()
    for resource in costs:
        resources.remove(resource)
    res_refs[index].set(resources)
    
    
def place(row, col, key, value):
    
    board = board_ref.get()
    board[row][col][key] = value
    board_ref.set(board)
    
    
def start(row, col, drow, dcol):
    
    place(row, col, 'town', current_index)
    place(row + drow, col + dcol, 'road', current_index)
    
    # TODO grant resources!
    

def road(row, col):

    take(current_index, [h, l])
    place(row, col, 'road', current_index)
    

def town(row, col):

    take(current_index, [h, l, g, w])
    place(row, col, 'town', current_index)
    

def city(row, col):

    take(current_index, [g, g, e, e, e])
    place(row, col, 'city', True)
    

def draw():

    take(current_index, [g, w, e])
    
    stack = stack_ref.get()
    card_index = random.randrange(len(stack))
    card = stack.pop(card_index)
    stack_ref.set(stack)

    cards = card_refs[current_index].get() or []
    cards.append(card)
    card_refs[current_index].set(cards)
    
    
def bandit(row, col):

    board = board_ref.get()
    for r in range(row_count):
        for c in range(col_count):
            board[r][c]['bandit'] = r == row and c == col
    board_ref.set(board)
    
    for index, res_ref in enumerate(res_refs):
        resources = res_ref.get()
        res_count = len(resources)
        if len(resources) > 7:
            name = player_names[index]
            drop_count = res_count // 2
            print(f"{name} [{index}] must drop {drop_count} resources!")
    

def steal(from_index):

    resources = res_refs[from_index].get()
    steal_index = random.randrange(len(resources))
    resource = resources.pop(steal_index)
    res_refs[from_index].set(resources)
    
    give(current_index, [resource])
    
    
def knight(row, col, steal_index):
    
    cards = card_refs[current_index].get()
    cards.remove("Ritter")
    cards.append("Ritter (gespielt)")
    card_refs[current_index].set(cards)
    
    knight_count = cards.count("Ritter (gespielt)")
    turn = turn_ref.get()
    if knight_count > turn['maxKnightCount']:
        turn['maxKnightCount'] = knight_count
        turn['maxKnightIndex'] = current_index
        turn_ref.set(turn)
        
    bandit(row, col)
    steal(from_index)
    
    
def build(row1, col1, row2, col2):

    road(row1, col1)
    road(row2, col2)
    

def mono(res):

    gains = []
    
    for index, res_ref in enumerate(res_refs):
        if index == current_index:
            continue
        resources = res_ref.get()
        losses = [r for r in resources if r == res]
        take(index, losses)
        gains += losses
        
    give(current_index, gains)

        
def invent(res1, res2):
    
    give(current_index, [res1, res2])

In [ ]:
S = 0
D = 1
H = 2
N = 3

In [ ]:
current_index = (current_index + 1) % player_count

In [ ]:
current_index = (current_index - 1) % player_count

In [ ]:
turn_ref.update({'index': current_index})

In [ ]:
turn_ref.update({'index': current_index, 'roll': random.randint(1, 6) + random.randint(1, 6)})

In [ ]:
current_index = first_index + 1

In [ ]:
start(2, 8, -1, 0)

In [ ]:
start(2, 14, 0, -1)

In [ ]:
start(8, 10, 0, 1)

In [ ]:
give(H, [g, w, e])

In [ ]:
town(6, 16)

In [ ]:
road(2, 9)

In [ ]:
draw()

In [ ]:
info()

In [ ]:
turn_ref.update({'longestRoadIndex': current_index, 'longestRoadCount': 5})

Fix:
- backend
    - trade functions
    - bandit loss communicate
    - roll resource automation!
- frontend
    - knight count in header?
    - Gelb Kontrast!